<a href="https://colab.research.google.com/github/N534H699/MachineLearning_Learn/blob/main/GPT_FromScratch_TO_FINISH_(Simpler_LLMs_first).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-05-29 21:00:38--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.1’

input.txt.1         100%[===================>]   1.06M  --.-KB/s    in 0.05s   

2023-05-29 21:00:39 (23.3 MB/s) - ‘input.txt.1’ saved [1115394/1115394]



In [ ]:
!ls

input.txt  input.txt.1	sample_data


In [ ]:
!ls

input.txt  input.txt.1	sample_data


In [ ]:
with open('input.txt', 'r', encoding='utf-8') as f:
  text = f.read()

In [ ]:
text

In [ ]:
len(text)

1115394

In [ ]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [ ]:
# Unique Characters in our text. I.e. our "Vocabulary"

In [ ]:
# Now need Tokenize the Input (convert to raw text as string to some sequence of characters according to some vocab of possible elements)

In [ ]:
stoi = { ch: i for i, ch in enumerate(chars) }
itos = { i:ch for i, ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

In [ ]:
encode("HI there")

[20, 21, 1, 58, 46, 43, 56, 43]

In [ ]:
decode ([20, 21, 1, 58, 46, 43, 56, 43])

'HI there'

In [ ]:
# Character Level Tokenizer (very small code books and very simple encode/decode functions -- for simplicity)

In [ ]:
import torch
data = torch.tensor( encode(text), dtype=torch.long ) # Encode all Tiny Shakespear
print( data.shape, data.dtype )
print( data[:100]) # First 100 Elements of Data Tensor (identical to first 100 chars of Tiny Shakespear Encoded)

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [ ]:
n = int( 0.9 * len(data) ) # First 90% will be training, rest is validation
train_data = data[:n]
val_data = data[n:] # Last 10% (help determine what extent Model Overfitting)

In [ ]:
len(train_data)

1003854

In [ ]:
len(val_data)

111540

In [ ]:
# Won't send all text in DataSet at once, would be computationally expensive
# So, train chunks at a time (each with some maximum length, e.g. "Block Size" may be called "Context Length")

In [ ]:
block_size = 8 # First 8 Characters
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [ ]:
# Many examples packed in when Sample Data like this,
# ... Since all characters follow each other,
# ... Simultaneously training @ every one of the positions (e.g. 18... 18,47... 18,47,56...)
# ... This gives Context, such as in context of 18 and 47, 56 is next character

In [ ]:
x = train_data[:block_size] # Inputs to Transformer (first Block-size Chars)
y = train_data[1:block_size+1] # Offset by one since y is Label / Target (what's next)
for t in range(block_size):
  context = x[:t+1] # Always all characters in X up to t, including t
  target = y[t] # Always t'th character in Targets Array Y
  print(f"when input is {context} the target: {target}")

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [ ]:
# ^ is all the 8 examples hidden in the chunk of 9 characters we sampled from Training Set
# This range of 1 to Block Size is useful to train Model to predict with Context as little as 1

In [ ]:
# 18 Minute: Batch Dimension for Effiency (Mini Batches of mult Chunks of text stacked in single Tensor (multiple arrays) to keep GPUs busy by processing multiple chunks at once)

In [ ]:
import torch
torch.manual_seed(1337)

In [ ]:
batch_size = 4 # Number independent sequence we'll process in parallel (#indep seq processing every forward/backward pass of Transformer)
block_size = 8 # Maximum Context Length for Predictions

def get_batch( split ):
  # Generate a small Batch of Data of Inputs X and Targets Y
  data = train_data if split == 'train' else val_data
  ix = torch.randint( len(data) - block_size, (batch_size,) ) # Generate random positions to grab a chunk out of, done by generating batch_size # of random offsets. Since batch_size = 4, ix will be 4 numbers, randomly generated between 0 and (len(data)-block_size)
  # One Dimensional Tensors will become Stacked in a row as a 4 x 8 Tensor
  x = torch.stack( [data[i:i+block_size] for i in ix] )
  y = torch.stack( [data[i+1: i+block_size+1] for i in ix] ) # Y's are 1 after Xs, so offset by one
  return x,y

xb, yb = get_batch( 'train' )
# Each Row is a Chunk of Training Set
print('inputs:'); print(xb.shape); print(xb);

# Comes in to Transformer at end to create the Loss Function (giving correct Answer for every Position inside X)
# Targets are used to look up correct answers for prediction corrections (Loss adjustments) during training
print( 'targets:'); print(yb.shape); print(yb);

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])


In [ ]:
print(xb)

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
